# Model 4
## Pakete und Daten laden

In einem ersten Schritt laden wir die nöten Pakete und laden die Daten.

In [4]:
import pandas as pd
import stopwords
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
import sklearn
import re
import spacy
import numpy as np
import random
import torch
from bertopic.vectorizers import ClassTfidfTransformer
import os
import openpyxl
import optuna
from sklearn.cluster import KMeans
from bertopic.representation import MaximalMarginalRelevance
from pathlib import Path

target = Path("C:/Users/mhu/Documents/github/topic_model_it")
os.chdir(target)

data_model_4 = pd.read_csv("data/informatikkurse.csv")
data_model_4 = data_model_4[data_model_4['model_4'] == 1].copy()
data_model_4

C:\Users\mhu\AppData\Local\Temp\ipykernel_29640\973783511.py:25: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data_model_4 = pd.read_csv("data/informatikkurse.csv")


,Unnamed: 0,veranstaltung_id,veranstaltung_titel,kursbeschreibung,kursformat_original,sprache_original,lernziele,lernmethode,literatur,voraussetzungen,...,medizin_info,wirt_info,recht_info,geo_info,bau_info,base_info,model_1_b,model_2,model_3,model_4
7,8,1717992,Inf-FD-DiPro: Didaktik der Programmierung,NaN,Seminar,NaN,NaN,NaN,NaN,Anmeldung über die StudiDB .,...,0,0,0,0,0,1,1,1,0,1
17,18,1717898,Inf-FD-IUG: Informatikunterricht gestalten,NaN,Übung,NaN,NaN,NaN,NaN,Anmeldung über die StudiDB .,...,0,0,0,0,0,1,1,1,0,1
89,90,1717905,Inf-FD-DiPro: Didaktik der Programmierung,NaN,Seminar,NaN,NaN,NaN,NaN,Anmeldung über die StudiDB .,...,0,0,0,0,0,1,1,1,0,1
126,127,1717955,Inf-SP-2F: Softwareprojekt (2F),NaN,Übung,NaN,NaN,NaN,NaN,Anmeldung über die StudiDB .,...,0,0,0,0,0,1,1,1,0,1
142,143,1717970,Inf-FD-Kombi: Ein Themenbereich der Informatik...,NaN,Seminar,NaN,NaN,NaN,NaN,In dem Modul wird eine fachwissenschaftliche V...,...,0,0,0,0,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51046,51047,1693557,Exercise: Knowledge Discovery and Data Mining,NaN,Übung,Englisch,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,0,1
51047,51048,1693558,Übung zu: Informationssysteme,NaN,Übung,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,0,1
51048,51049,1693559,Übung zu: Informationssysteme,NaN,Übung,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,1,0,1
51049,51050,1693560,infKDDM-01a: Knowledge Discovery and Data Mining,Kurzfassung: Knowledge Discovery is the nontri...,Vorlesung,Englisch,NaN,NaN,There is no specific textbook the classs mate...,Voraussetzungen: Basic knowledge about data sc...,...,0,0,0,0,0,1,1,1,0,1


## Reproduzierbarkeit durch Seeds
Im folgenden fixieren wir die Zufallszahlen-Generatoren aller beteiligten Bibliotheken auf den Wert 11. Dies stellt sicher, dass die Experimente bei jedem Durchlauf identische Ergebnisse liefern.

In [5]:
seed = 11  # Initialisiert den Seed-Wert für reproduzierbare Ergebnisse
np.random.seed(seed)  # Setzt den Seed für NumPy-Zufallszahlengeneratoren
random.seed(seed)  # Setzt den Seed für den Python-eigenen Zufallszahlengenerator
torch.manual_seed(seed)  # Setzt den Seed für PyTorch-Zufallszahlen
if torch.cuda.is_available():  # Überprüft, ob CUDA (GPU-Unterstützung) verfügbar ist
    torch.cuda.manual_seed_all(seed)  # Setzt den Seed für alle CUDA-Zufallszahlen (für GPU-Berechnungen)

## Docs

Wir generieren als nächsten die Dokumente, die das Eingangsmaterial für das Topic-Model bilden.

In [6]:
# Anzeige der Spaltennamen von data_model_4
print(data_model_4.columns.tolist())

data_model_4 = data_model_4[["veranstaltung_titel", "kursbeschreibung"]].copy()
data_model_4.head()

data_model_4 = data_model_4.apply(lambda x: x.fillna('') if x.dtype == 'O' else x)  # Ersetzt fehlende Werte durch leere Strings in Objektspalten (Strings) und belässt numerische Spalten unverändert
data_model_4['titel_kursbesch'] = data_model_4['veranstaltung_titel'] + ' ' + data_model_4['kursbeschreibung']  # Kombiniert die Spalten "titel" und "kursbeschreibung" zu einer neuen Spalte "titel_kursbesch"
docs = data_model_4['titel_kursbesch'].tolist()  # Konvertiert die Inhalte der Spalte "titel_kursbesch" in eine Liste von Strings

['Unnamed: 0', 'veranstaltung_id', 'veranstaltung_titel', 'kursbeschreibung', 'kursformat_original', 'sprache_original', 'lernziele', 'lernmethode', 'literatur', 'voraussetzungen', 'zusatzinformationen', 'anmerkungen', 'pruefung', 'dozierende', 'teilnehmerzahl', 'sws', 'ects', 'url', 'nummer', 'pfad', 'scrape_datum', 'hochschule_id', 'hochschule_name', 'fakultaet_id', 'fakultaet_name', 'organisation_id', 'organisation_name', 'semester_id', 'semester_name', 'sprache_id', 'sprache_name', 'kursformat_id', 'kursformat_name', 'matchingart_id', 'matchingart_name', 'lehr_und_forschungsbereich_id', 'bio_info', 'ingen_info', 'medien_info', 'medizin_info', 'wirt_info', 'recht_info', 'geo_info', 'bau_info', 'base_info', 'model_1_b', 'model_2', 'model_3', 'model_4']


## Stopwords

In [9]:
from utils import stopwords_config

irrelevant_terms = stopwords_config.irrelevant_terms

sw = list(stopwords.get_stopwords("en"))
sw.extend(list(stopwords.get_stopwords("de")))
sw.extend(irrelevant_terms)
irrelevant_terms

['vl',
 'übung',
 'übungen',
 'seminar',
 'arbeitsgruppenseminar',
 'oberseminar',
 'proseminar',
 'blockveranstaltung',
 'vorlesung',
 'kolloquium',
 'theoriekolloquium',
 'einführung',
 'tutorium',
 'ue',
 'vereinbarung',
 'projekt',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',
 'begleitseminar',
 'abschlussarbeiten',
 'unterrichtspraktikum',
 'masterseminar',
 'proseminare',
 'praxisseminar',
 'praxissemester',
 'schulpraxis',
 'ringpraktikum',
 'basispraktikum',
 'praxistage',
 'industriepraktikum',
 'vorkurs',
 'projektseminar',
 'juniorprofessur',
 'masterarbeiten',
 'forschungsseminar',
 'modulbeschreibung',
 'veranstaltung',
 'kommentare',
 'raum',
 'uhrzeit',
 'vereinbarung',
 'vorlesung',
 'praktikum',
 'masterprojekt',
 'wiederholerklausur',
 'fortgeschrittenenpraktikum',
 'hauptseminar',
 'fachpraktikum',
 'ergänzungsvorlesung',
 'forschungspraktikum',

# Model

Wir definieren unser Model.

In [10]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("CUDA available:", torch.cuda.is_available())
print("Current device:", model.device)

CUDA available: True
Current device: cuda:0


## Model-Settings, GPU-Check
Konfiguration frei wählbar (einfach im Code unten anpassen).
Hier kann eine erste explorative Untersuchung durchgeführt werden.

Wir checken weiterhin, ob wirklich unsere GPU verwendet wird. Wenn `cuda:0` ist dies der Fall.

In [11]:
# CountVectorizer
vectorizer = CountVectorizer(
  stop_words=sw,  # Entfernt Stopwörter basierend auf der angegebenen Liste (sw)
  token_pattern=r'\b\w+\b',  # Extrahiert nur ganze Wörter, d. h. keine Sonderzeichen oder Zahlen
  ngram_range=(1, 3)  # Erstellt 1-Gramme (einzelne Wörter) bis 3-Gramme (Wortgruppen aus bis zu 3 aufeinanderfolgenden Wörtern)
)

# Embedding Settings  
embedding_model = SentenceTransformer(
    "paraphrase-multilingual-mpnet-base-v2",
    device="cuda" if torch.cuda.is_available() else "cpu"
)

# UMAP Settings
umap_model = UMAP(
    n_neighbors=10,
    n_components=10,
    min_dist=0.0,
    metric="cosine",
    random_state=seed
)
# HDBSCAN Settings
hdbscan_model = HDBSCAN(
    min_cluster_size=100,
    cluster_selection_epsilon=0.2,
    prediction_data=True
)
# Representation Settings
representation_model = MaximalMarginalRelevance(diversity=0.1)

# BERTopic initialisieren
data_model_4 = BERTopic(
  embedding_model=embedding_model,
  # min_topic_size=100,
  nr_topics="auto", 
  language="multilingual",
  verbose=True,
  umap_model=umap_model,
  vectorizer_model=vectorizer,
  hdbscan_model=hdbscan_model,
  top_n_words = 20,
  representation_model=representation_model
)

## Training

Wir trainieren das Modell.

In [12]:
data_model_4.fit(docs)
data_model_4.get_topic_info()

2026-02-05 16:01:08,145 - BERTopic - Embedding - Transforming documents to embeddings.
Batches: 100%|██████████| 157/157 [00:03<00:00, 42.48it/s] 
2026-02-05 16:01:11,892 - BERTopic - Embedding - Completed ✓
2026-02-05 16:01:11,892 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2026-02-05 16:01:24,884 - BERTopic - Dimensionality - Completed ✓
2026-02-05 16:01:24,885 - BERTopic - Cluster - Start clustering the reduced embeddings
2026-02-05 16:01:25,067 - BERTopic - Cluster - Completed ✓
2026-02-05 16:01:25,067 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2026-02-05 16:01:25,469 - BERTopic - Representation - Completed ✓
2026-02-05 16:01:25,470 - BERTopic - Topic reduction - Reducing number of topics
2026-02-05 16:01:25,475 - BERTopic - Representation - Fine-tuning topics using representation models.
2026-02-05 16:01:26,211 - BERTopic - Representation - Completed ✓
2026-02-05 16:01:26,213 - BERTopic - Topic reduction - R

,Topic,Count,Name,Representation,Representative_Docs
0,-1,2024,-1_informatik_data_systems_systeme,"[informatik, data, systems, systeme, modellier...",[Praktikum DBMS: Ubiquitous Texttechnologies I...
1,0,624,0_informatik_01_data_wissenschaftlichen,"[informatik, 01, data, wissenschaftlichen, inf...","[Informatik-Kolloquium , Übung zu Informatik A..."
2,1,324,1_software_engineering_software engineering_so...,"[software, engineering, software engineering, ...","[Software-Projektseminar , Software-Praktikum ..."
3,2,316,2_learning_robotik_machine learning_roboy,"[learning, robotik, machine learning, roboy, d...","[Machine Learning , Praktikum - Roboy Student ..."
4,3,261,3_management_wirtschaftsinformatik_outsourcing...,"[management, wirtschaftsinformatik, outsourcin...",[WInf-InfWiMa: Informations- und Wissensmanage...
5,4,224,4_security_secure_cryptography_security privacy,"[security, secure, cryptography, security priv...",[Bachelor/Master-Seminar - Usable Security and...
6,5,205,5_algorithmen_algorithms_algorithmen datenstru...,"[algorithmen, algorithms, algorithmen datenstr...",[Effiziente Algorithmen 1 Ein zentrales Proble...
7,6,205,6_bildverarbeitung_3d_visualization_graphics,"[bildverarbeitung, 3d, visualization, graphics...",[VCP-PR Praktikum Visual Computing Projekt-Pra...
8,7,197,7_bioinformatik_fuzzy_fuzzy methoden_protein,"[bioinformatik, fuzzy, fuzzy methoden, protein...",[Praktische Übung zu: Fuzzy-Methoden Kurzfassu...
9,8,183,8_internet_rechnerarchitektur_computing_kommun...,"[internet, rechnerarchitektur, computing, komm...",[Systemprogrammierung In diesem Praktikum werd...


## Topic Visualiszieren, Outlier Reduzieren, Topics mergen

In [13]:
data_model_4.visualize_topics()
#data_model_4.visualize_hierarchy() # Hilft extrem zu sehen, welche Themen zusammengehören

In [ ]:
# topic_model.merge_topics(docs, [0, 13])

# # Outlier reduzieren
# # BERTopic auf Test-Daten anwenden
# topics, probs = data_model_4.transform(docs)
# print(data_model_4.get_topic_freq())

# # # Outlier reduzieren
# topics = data_model_4.reduce_outliers(docs, topics)
# data_model_4.update_topics(docs, topics=topics)

In [ ]:
data_model_4.get_topic_info()

## Labeling

Mit Hilfe von gpt4o lassen wir uns die Topcis labeln. Das erleichtert die Interpretation des Modells

In [14]:
from openai import OpenAI as OpenAIClient
from bertopic.representation import OpenAI as OpenAIRep

client = OpenAIClient()  # nutzt OPENAI_API_KEY aus der Umgebung
rep = OpenAIRep(
    client=client, 
    model="gpt-4o-mini", 
    delay_in_seconds=10,      # Erhöhe Delay
    exponential_backoff=True, # Aktiviere das schrittweise längere Warten
    nr_docs=3                 # Weniger Dokumente pro Topic reduzieren die Token-Last
)
data_model_4.update_topics(docs, representation_model=rep)
data_model_4.get_topic_info() 

100%|██████████| 13/13 [02:22<00:00, 10.94s/it]


,Topic,Count,Name,Representation,Representative_Docs
0,-1,2024,-1_Human-Computer Interaction,[Human-Computer Interaction],[Praktikum DBMS: Ubiquitous Texttechnologies I...
1,0,624,0_Scientific Work in Informatics,[Scientific Work in Informatics],"[Informatik-Kolloquium , Übung zu Informatik A..."
2,1,324,1_Software Engineering Project,[Software Engineering Project],"[Software-Projektseminar , Software-Praktikum ..."
3,2,316,2_Robotics and Deep Learning,[Robotics and Deep Learning],"[Machine Learning , Praktikum - Roboy Student ..."
4,3,261,3_Wirtschaftsinformatik und IT-Management,[Wirtschaftsinformatik und IT-Management],[WInf-InfWiMa: Informations- und Wissensmanage...
5,4,224,4_Economics of Security and Privacy,[Economics of Security and Privacy],[Bachelor/Master-Seminar - Usable Security and...
6,5,205,5_Efficient Algorithms Design,[Efficient Algorithms Design],[Effiziente Algorithmen 1 Ein zentrales Proble...
7,6,205,6_Text Imaging Technologies,[Text Imaging Technologies],[VCP-PR Praktikum Visual Computing Projekt-Pra...
8,7,197,7_Fuzzy methods in education,[Fuzzy methods in education],[Praktische Übung zu: Fuzzy-Methoden Kurzfassu...
9,8,183,8_Grundlagen der Hardwareinformatik,[Grundlagen der Hardwareinformatik],[Systemprogrammierung In diesem Praktikum werd...


In [ ]:
# Speichern
topic_model.save("models/explor_model_1", serialization="safetensors", save_embedding_model=True)

# Laden
from bertopic import BERTopic
loaded_model = BERTopic.load("models/explor_model_1")